In [1]:
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

device = "cuda" if torch.cuda.is_available() else "cpu"

model = vgg16(pretrained=True) #vgg16 모델 객체 생성, ImageNet 데이터셋으로 이미 학습된 것을 가져와 전이학습
#초기 가중치의 성능이 좋은 상태이므로, 적은 데이터로도 빠르게 학습할 수 있음

fc = nn.Sequential( #분류층을 새롭게 정의함. ImageNet  데이터가 아니라, CIFAR-10 데이터이기 때문에 마지막 레이어는 10개로 나오게 해야함
       nn.Linear(512 * 7 * 7, 4096),
       nn.ReLU(),
       nn.Dropout(), #드롭아웃층 정의
       nn.Linear(4096, 4096),
       nn.ReLU(),
       nn.Dropout(),
       nn.Linear(4096, 10),
   )
"""
(Flatten된 입력: 512 * 7 * 7 = 25088) → FC(4096) → ReLU → Dropout
→ FC(4096) → ReLU → Dropout
→ FC(10) → 출력
"""
model.classifier = fc #VGG의 classifier 부분을 우리가 만든 것으로 덮어씀
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 95.8MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [2]:
import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader

from torch.optim.adam import Adam

#cifar-10 데이터셋의 이미지 크기는 32*32인데, ImageNet은 224*224임. 전이학습된 걸 사용하기 위해 이미지를 resize함
train_transforms = Compose([
   Resize(224),
   RandomCrop((224, 224), padding=4),#resize 후 crop을 권장함. crop 후 resize하면 정보 손실이 크기 때문.
   RandomHorizontalFlip(p=0.5),
   ToTensor(),
   Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))#이전 실습에서 얻은 값..
])

In [3]:
#test데이터용 전처리
test_transforms = Compose([
    Resize(224),  # 또는 Resize((224, 224))도 가능
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

In [4]:
training_data = CIFAR10(root="./", train=True, download=True, transform=train_transforms)
test_data = CIFAR10(root="./", train=False, download=True, transform=test_transforms)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

100%|██████████| 170M/170M [00:05<00:00, 30.9MB/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [5]:
lr = 1e-4 #전이학습에서는 보통 학습률을 적게 설정함. 1e-4, 1e-5 ...
optim = Adam(model.parameters(), lr=lr)

for epoch in range(1):#epoch 수정하기 -> 예: 5
   iterator = tqdm.tqdm(train_loader) #학습 진행바 보이도록 출
   for data, label in iterator:
       optim.zero_grad()#매 batch마다 gradient 초기화(전이학습 된 가중치가 초기화되는 게 아님!)
       #gradient를 초기화하지 않으면  gradient가 배치마다 누적되기 때문에 원치 않는 결과가 나옴.

       preds = model(data.to(device)) # 모델의 예측값 출력

       loss = nn.CrossEntropyLoss()(preds, label.to(device))#손실 계산
       loss.backward()#역전파
       optim.step()#가중치 업데이트

       # tqdm이 출력할 문자열 지정
       iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "CIFAR_pretrained.pth") # 모델 저장

epoch:1 loss:0.2635610103607178: 100%|██████████| 1563/1563 [14:16<00:00,  1.83it/s]


In [6]:
model.load_state_dict(torch.load("CIFAR_pretrained.pth", map_location=device))

num_corr = 0

with torch.no_grad():
   for data, label in test_loader:

       output = model(data.to(device))
       preds = output.data.max(1)[1]
       corr = preds.eq(label.to(device).data).sum().item()
       num_corr += corr

   print(f"Accuracy:{num_corr/len(test_data)}")

<ipython-input-6-1f03128bc2c8>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("CIFAR_pretrained.pth", map_location=device))


Accuracy:0.8934
